# Data set preparation

In [72]:
import sys
# import scipy as sc
# from scipy import stats
# import numpy as np
import pandas as pd
# import datetime as dt
import math
import pickle
# import scipy.cluster.hierarchy as shc
from tqdm import tqdm
# from alignment.sequencealigner import SimpleScoring, GlobalSequenceAligner
# from alignment.vocabulary import Vocabulary
# from alignment.sequence import Sequence


# sys.setrecursionlimit(5000000)


## Load individual sets

In [56]:

metadata = pd.read_csv("./dataset_files/mimic-cxr-2.0.0-metadata.csv", index_col=0)
# icu_binned = pd.read_csv('./dataset_files/icu_binned_v2.csv', low_memory=False, index_col=0)
all_stays = pd.read_csv('./dataset_files/all_stays.csv', index_col=0)

all_stays['charttime'] = pd.to_datetime(all_stays['intime'])
# icu_binned['charttime'] = pd.to_datetime(icu_binned['charttime'], format='%Y-%m-%d')


## Check ICU binned

In [3]:
icu_binned.head()

,subject_id,hadm_id,stay_id,charttime,storetime,itemid,value,valuenum,valueuom,warning,label,abbreviation,linksto,category,param_type,lownormalvalue,highnormalvalue,value_categorical
0,10582595,20690213,33235984,2110-02-09 20:42:00,2110-02-10 16:45:00,226512,54.3,54.3,kg,0,Admission Weight (Kg),Admission Weight (Kg),chartevents,General,Numeric,NaN,NaN,7036
7227,10264572,25328682,34149732,2110-02-17 01:40:00,2110-02-17 03:45:00,226512,78,78.0,kg,0,Admission Weight (Kg),Admission Weight (Kg),chartevents,General,Numeric,NaN,NaN,7969
12934,10617255,21586668,37228716,2110-06-17 23:30:00,2110-06-18 01:14:00,226512,61.1,61.1,kg,0,Admission Weight (Kg),Admission Weight (Kg),chartevents,General,Numeric,NaN,NaN,7231
13690,10499563,27896354,32084946,2110-11-09 06:30:00,2110-11-09 07:30:00,226512,128,128.0,kg,0,Admission Weight (Kg),Admission Weight (Kg),chartevents,General,Numeric,NaN,NaN,3346
16190,10168722,25403630,32404106,2110-11-14 18:21:00,2110-11-14 21:18:00,226512,53.6,53.6,kg,0,Admission Weight (Kg),Admission Weight (Kg),chartevents,General,Numeric,NaN,NaN,6796


In [4]:
icu_binned[icu_binned['value_categorical'] == 0]

,subject_id,hadm_id,stay_id,charttime,storetime,itemid,value,valuenum,valueuom,warning,label,abbreviation,linksto,category,param_type,lownormalvalue,highnormalvalue,value_categorical
1813156,10862862,20640606,38991873,2133-05-25 00:00:00,2133-05-25 02:06:00,224860,NaN,NaN,NaN,0,Skin Condition,Skin Condition,chartevents,Skin - Assessment,Text,NaN,NaN,0
1813352,10862862,20640606,38991873,2133-05-25 04:00:00,2133-05-25 04:51:00,224860,NaN,NaN,NaN,0,Skin Condition,Skin Condition,chartevents,Skin - Assessment,Text,NaN,NaN,0
2907365,10227155,27191372,34728328,2144-09-11 08:00:00,2144-09-11 12:58:00,229534,NaN,NaN,NaN,0,Multi Lumen - Obturator in place,Multi Lumen - Obturator in place,chartevents,Access Lines - Invasive,Text,NaN,NaN,0
9383889,10670085,26051661,35913126,2199-05-17 12:00:00,2199-05-17 15:31:00,224860,NaN,NaN,NaN,0,Skin Condition,Skin Condition,chartevents,Skin - Assessment,Text,NaN,NaN,0


## Check ICU data

In [67]:
def get_distribution(dataframe):
    
    multiple_values = []
    single_values = []
    types = []
    values = list(dataframe.value_categorical.unique())
    
    for value in tqdm(values):
        reverse_mapping = dataframe[dataframe['value_categorical'] == value]
        labels = reverse_mapping.label.unique()

        if (len(labels) > 1):
            multiple_values.append(value)

        else:
            single_values.append(value)
        
    for label in tqdm(multiple_values):
        types += list(dataframe[dataframe['value_categorical']
                                == label].param_type.unique())

    print(
        f"Multiple values: {len(multiple_values)}\nSingle values: {len(single_values)} \nValue types: {list(dict.fromkeys(types))}")


In [ ]:
get_distribution(icu_binned)

In [49]:
icu_text = pd.read_csv('./dataset_files/text_df.csv', index_col=0)
icu_binned = pd.concat([icu_binned[icu_binned['param_type'] != 'Text'], icu_text])
icu_binned.head()

,subject_id,hadm_id,stay_id,charttime,storetime,itemid,value,valuenum,valueuom,warning,label,abbreviation,linksto,category,param_type,lownormalvalue,highnormalvalue,value_categorical,Unnamed: 0
0,10582595,20690213,33235984,2110-02-09 20:42:00,2110-02-10 16:45:00,226512,54.3,54.3,kg,0,Admission Weight (Kg),Admission Weight (Kg),chartevents,General,Numeric,NaN,NaN,7036,NaN
7227,10264572,25328682,34149732,2110-02-17 01:40:00,2110-02-17 03:45:00,226512,78,78.0,kg,0,Admission Weight (Kg),Admission Weight (Kg),chartevents,General,Numeric,NaN,NaN,7969,NaN
12934,10617255,21586668,37228716,2110-06-17 23:30:00,2110-06-18 01:14:00,226512,61.1,61.1,kg,0,Admission Weight (Kg),Admission Weight (Kg),chartevents,General,Numeric,NaN,NaN,7231,NaN
13690,10499563,27896354,32084946,2110-11-09 06:30:00,2110-11-09 07:30:00,226512,128,128.0,kg,0,Admission Weight (Kg),Admission Weight (Kg),chartevents,General,Numeric,NaN,NaN,3346,NaN
16190,10168722,25403630,32404106,2110-11-14 18:21:00,2110-11-14 21:18:00,226512,53.6,53.6,kg,0,Admission Weight (Kg),Admission Weight (Kg),chartevents,General,Numeric,NaN,NaN,6796,NaN


In [52]:
get_distribution(icu_binned)

Multiple values: 4071
Single values: 4730 
Value types: ['Numeric', 'Text', 'Checkbox', 'Numeric with tag']


## Encode metadata

In [91]:
metadata.head()


,subject_id,study_id,PerformedProcedureStepDescription,ViewPosition,Rows,Columns,StudyDate,StudyTime,ProcedureCodeSequence_CodeMeaning,ViewCodeSequence_CodeMeaning,PatientOrientationCodeSequence_CodeMeaning
dicom_id,,,,,,,,,,,
02aa804e-bde0afdd-112c0b34-7bc16630-4e384014,10000032,50414267,CHEST (PA AND LAT),PA,3056,2544,21800506,213014.531,CHEST (PA AND LAT),postero-anterior,Erect
174413ec-4ec4c1f7-34ea26b7-c5f994f8-79ef1962,10000032,50414267,CHEST (PA AND LAT),LATERAL,3056,2544,21800506,213014.531,CHEST (PA AND LAT),lateral,Erect
2a2277a9-b0ded155-c0de8eb9-c124d10e-82c5caab,10000032,53189527,CHEST (PA AND LAT),PA,3056,2544,21800626,165500.312,CHEST (PA AND LAT),postero-anterior,Erect
e084de3b-be89b11e-20fe3f9f-9c8d8dfe-4cfd202c,10000032,53189527,CHEST (PA AND LAT),LATERAL,3056,2544,21800626,165500.312,CHEST (PA AND LAT),lateral,Erect
68b5c4b1-227d0485-9cc38c3f-7b84ab51-4b472714,10000032,53911762,CHEST (PORTABLE AP),AP,2705,2539,21800723,80556.875,CHEST (PORTABLE AP),antero-posterior,NaN


In [92]:
metadata['value'] = 'photo ' + metadata['PerformedProcedureStepDescription'].astype(str)
metadata['value'] = metadata['value'].astype('category')
metadata['value_categorical'] = metadata['value'].cat.codes
metadata['param_type'] = 'ED photo'
metadata.head()


,subject_id,study_id,PerformedProcedureStepDescription,ViewPosition,Rows,Columns,StudyDate,StudyTime,ProcedureCodeSequence_CodeMeaning,ViewCodeSequence_CodeMeaning,PatientOrientationCodeSequence_CodeMeaning,value,value_categorical,param_type
dicom_id,,,,,,,,,,,,,,
02aa804e-bde0afdd-112c0b34-7bc16630-4e384014,10000032,50414267,CHEST (PA AND LAT),PA,3056,2544,21800506,213014.531,CHEST (PA AND LAT),postero-anterior,Erect,photo CHEST (PA AND LAT),14,ED photo
174413ec-4ec4c1f7-34ea26b7-c5f994f8-79ef1962,10000032,50414267,CHEST (PA AND LAT),LATERAL,3056,2544,21800506,213014.531,CHEST (PA AND LAT),lateral,Erect,photo CHEST (PA AND LAT),14,ED photo
2a2277a9-b0ded155-c0de8eb9-c124d10e-82c5caab,10000032,53189527,CHEST (PA AND LAT),PA,3056,2544,21800626,165500.312,CHEST (PA AND LAT),postero-anterior,Erect,photo CHEST (PA AND LAT),14,ED photo
e084de3b-be89b11e-20fe3f9f-9c8d8dfe-4cfd202c,10000032,53189527,CHEST (PA AND LAT),LATERAL,3056,2544,21800626,165500.312,CHEST (PA AND LAT),lateral,Erect,photo CHEST (PA AND LAT),14,ED photo
68b5c4b1-227d0485-9cc38c3f-7b84ab51-4b472714,10000032,53911762,CHEST (PORTABLE AP),AP,2705,2539,21800723,80556.875,CHEST (PORTABLE AP),antero-posterior,NaN,photo CHEST (PORTABLE AP),17,ED photo


In [135]:
metadata['value_categorical'] = 'metadata ' + metadata['value_categorical'].astype(str)
metadata.head()

,subject_id,study_id,PerformedProcedureStepDescription,ViewPosition,Rows,Columns,StudyDate,StudyTime,ProcedureCodeSequence_CodeMeaning,ViewCodeSequence_CodeMeaning,PatientOrientationCodeSequence_CodeMeaning,value,value_categorical,param_type,charttime
dicom_id,,,,,,,,,,,,,,,
02aa804e-bde0afdd-112c0b34-7bc16630-4e384014,10000032,50414267,CHEST (PA AND LAT),PA,3056,2544,21800506,213014.531,CHEST (PA AND LAT),postero-anterior,Erect,photo CHEST (PA AND LAT),metadata 14,ED photo,2180-05-06
174413ec-4ec4c1f7-34ea26b7-c5f994f8-79ef1962,10000032,50414267,CHEST (PA AND LAT),LATERAL,3056,2544,21800506,213014.531,CHEST (PA AND LAT),lateral,Erect,photo CHEST (PA AND LAT),metadata 14,ED photo,2180-05-06
2a2277a9-b0ded155-c0de8eb9-c124d10e-82c5caab,10000032,53189527,CHEST (PA AND LAT),PA,3056,2544,21800626,165500.312,CHEST (PA AND LAT),postero-anterior,Erect,photo CHEST (PA AND LAT),metadata 14,ED photo,2180-06-26
e084de3b-be89b11e-20fe3f9f-9c8d8dfe-4cfd202c,10000032,53189527,CHEST (PA AND LAT),LATERAL,3056,2544,21800626,165500.312,CHEST (PA AND LAT),lateral,Erect,photo CHEST (PA AND LAT),metadata 14,ED photo,2180-06-26
68b5c4b1-227d0485-9cc38c3f-7b84ab51-4b472714,10000032,53911762,CHEST (PORTABLE AP),AP,2705,2539,21800723,80556.875,CHEST (PORTABLE AP),antero-posterior,NaN,photo CHEST (PORTABLE AP),metadata 17,ED photo,2180-07-23


In [95]:
metadata.head()

,subject_id,study_id,PerformedProcedureStepDescription,ViewPosition,Rows,Columns,StudyDate,StudyTime,ProcedureCodeSequence_CodeMeaning,ViewCodeSequence_CodeMeaning,PatientOrientationCodeSequence_CodeMeaning,value,value_categorical,param_type
dicom_id,,,,,,,,,,,,,,
02aa804e-bde0afdd-112c0b34-7bc16630-4e384014,10000032,50414267,CHEST (PA AND LAT),PA,3056,2544,21800506,213014.531,CHEST (PA AND LAT),postero-anterior,Erect,photo CHEST (PA AND LAT),14,ED photo
174413ec-4ec4c1f7-34ea26b7-c5f994f8-79ef1962,10000032,50414267,CHEST (PA AND LAT),LATERAL,3056,2544,21800506,213014.531,CHEST (PA AND LAT),lateral,Erect,photo CHEST (PA AND LAT),14,ED photo
2a2277a9-b0ded155-c0de8eb9-c124d10e-82c5caab,10000032,53189527,CHEST (PA AND LAT),PA,3056,2544,21800626,165500.312,CHEST (PA AND LAT),postero-anterior,Erect,photo CHEST (PA AND LAT),14,ED photo
e084de3b-be89b11e-20fe3f9f-9c8d8dfe-4cfd202c,10000032,53189527,CHEST (PA AND LAT),LATERAL,3056,2544,21800626,165500.312,CHEST (PA AND LAT),lateral,Erect,photo CHEST (PA AND LAT),14,ED photo
68b5c4b1-227d0485-9cc38c3f-7b84ab51-4b472714,10000032,53911762,CHEST (PORTABLE AP),AP,2705,2539,21800723,80556.875,CHEST (PORTABLE AP),antero-posterior,NaN,photo CHEST (PORTABLE AP),17,ED photo


In [134]:
metadata['charttime'] = pd.to_datetime(metadata['StudyDate'], format='%Y%m%d')
metadata.head()

,subject_id,study_id,PerformedProcedureStepDescription,ViewPosition,Rows,Columns,StudyDate,StudyTime,ProcedureCodeSequence_CodeMeaning,ViewCodeSequence_CodeMeaning,PatientOrientationCodeSequence_CodeMeaning,value,value_categorical,param_type,charttime
dicom_id,,,,,,,,,,,,,,,
02aa804e-bde0afdd-112c0b34-7bc16630-4e384014,10000032,50414267,CHEST (PA AND LAT),PA,3056,2544,21800506,213014.531,CHEST (PA AND LAT),postero-anterior,Erect,photo CHEST (PA AND LAT),14,ED photo,2180-05-06
174413ec-4ec4c1f7-34ea26b7-c5f994f8-79ef1962,10000032,50414267,CHEST (PA AND LAT),LATERAL,3056,2544,21800506,213014.531,CHEST (PA AND LAT),lateral,Erect,photo CHEST (PA AND LAT),14,ED photo,2180-05-06
2a2277a9-b0ded155-c0de8eb9-c124d10e-82c5caab,10000032,53189527,CHEST (PA AND LAT),PA,3056,2544,21800626,165500.312,CHEST (PA AND LAT),postero-anterior,Erect,photo CHEST (PA AND LAT),14,ED photo,2180-06-26
e084de3b-be89b11e-20fe3f9f-9c8d8dfe-4cfd202c,10000032,53189527,CHEST (PA AND LAT),LATERAL,3056,2544,21800626,165500.312,CHEST (PA AND LAT),lateral,Erect,photo CHEST (PA AND LAT),14,ED photo,2180-06-26
68b5c4b1-227d0485-9cc38c3f-7b84ab51-4b472714,10000032,53911762,CHEST (PORTABLE AP),AP,2705,2539,21800723,80556.875,CHEST (PORTABLE AP),antero-posterior,NaN,photo CHEST (PORTABLE AP),17,ED photo,2180-07-23


## Encode stays

In [98]:
all_stays.head()


,subject_id,hadm_id,transfer_id,eventtype,careunit,intime,outtime
0,10264572,25328682,31523226,ED,Emergency Department,2110-02-17 00:03:00,2110-02-17 03:46:00
1,10264572,25328682,34149732,admit,Trauma SICU (TSICU),2110-02-17 03:46:00,2110-02-21 03:24:17
2,10264572,25328682,35175138,transfer,Vascular,2110-02-21 03:24:17,2110-03-01 14:18:25
3,10264572,25328682,30570317,transfer,Vascular,2110-03-01 14:18:25,2110-03-04 14:17:51
4,10264572,25328682,37295469,discharge,NaN,2110-03-04 14:17:51,NaN


In [138]:
all_stays['value'] = 'all_stays ' + \
    all_stays['careunit'].astype(str)
all_stays['value'] = all_stays['value'].astype('category')
all_stays['value_categorical'] = all_stays['value'].cat.codes
all_stays['param_type'] = 'Transfer'
all_stays.head()


,subject_id,hadm_id,transfer_id,eventtype,careunit,intime,outtime,charttime,value,value_categorical,param_type
0,10264572,25328682,31523226,ED,Emergency Department,2110-02-17 00:03:00,2110-02-17 03:46:00,2110-02-17 00:03:00,all_stays Emergency Department,3,Transfer
1,10264572,25328682,34149732,admit,Trauma SICU (TSICU),2110-02-17 03:46:00,2110-02-21 03:24:17,2110-02-17 03:46:00,all_stays Trauma SICU (TSICU),22,Transfer
2,10264572,25328682,35175138,transfer,Vascular,2110-02-21 03:24:17,2110-03-01 14:18:25,2110-02-21 03:24:17,all_stays Vascular,24,Transfer
3,10264572,25328682,30570317,transfer,Vascular,2110-03-01 14:18:25,2110-03-04 14:17:51,2110-03-01 14:18:25,all_stays Vascular,24,Transfer
4,10264572,25328682,37295469,discharge,NaN,2110-03-04 14:17:51,NaN,2110-03-04 14:17:51,all_stays nan,25,Transfer


In [128]:
all_stays['value_categorical'].isnull().values.any()


False

In [139]:
all_stays['value_categorical'] = 'all_stays ' + all_stays['value_categorical'].astype(str)
all_stays.head()


,subject_id,hadm_id,transfer_id,eventtype,careunit,intime,outtime,charttime,value,value_categorical,param_type
0,10264572,25328682,31523226,ED,Emergency Department,2110-02-17 00:03:00,2110-02-17 03:46:00,2110-02-17 00:03:00,all_stays Emergency Department,all_stays 3,Transfer
1,10264572,25328682,34149732,admit,Trauma SICU (TSICU),2110-02-17 03:46:00,2110-02-21 03:24:17,2110-02-17 03:46:00,all_stays Trauma SICU (TSICU),all_stays 22,Transfer
2,10264572,25328682,35175138,transfer,Vascular,2110-02-21 03:24:17,2110-03-01 14:18:25,2110-02-21 03:24:17,all_stays Vascular,all_stays 24,Transfer
3,10264572,25328682,30570317,transfer,Vascular,2110-03-01 14:18:25,2110-03-04 14:17:51,2110-03-01 14:18:25,all_stays Vascular,all_stays 24,Transfer
4,10264572,25328682,37295469,discharge,NaN,2110-03-04 14:17:51,NaN,2110-03-04 14:17:51,all_stays nan,all_stays 25,Transfer


In [136]:
all_stays['charttime'] = pd.to_datetime(all_stays['intime'])
all_stays.head()


,subject_id,hadm_id,transfer_id,eventtype,careunit,intime,outtime,charttime
0,10264572,25328682,31523226,ED,Emergency Department,2110-02-17 00:03:00,2110-02-17 03:46:00,2110-02-17 00:03:00
1,10264572,25328682,34149732,admit,Trauma SICU (TSICU),2110-02-17 03:46:00,2110-02-21 03:24:17,2110-02-17 03:46:00
2,10264572,25328682,35175138,transfer,Vascular,2110-02-21 03:24:17,2110-03-01 14:18:25,2110-02-21 03:24:17
3,10264572,25328682,30570317,transfer,Vascular,2110-03-01 14:18:25,2110-03-04 14:17:51,2110-03-01 14:18:25
4,10264572,25328682,37295469,discharge,NaN,2110-03-04 14:17:51,NaN,2110-03-04 14:17:51


## Merge datasets

In [112]:
complete_dataset = pd.concat([all_stays, metadata, icu_binned])
complete_dataset.head()

,subject_id,hadm_id,transfer_id,eventtype,careunit,intime,outtime,value,value_categorical,param_type,...,itemid,valuenum,valueuom,warning,label,abbreviation,linksto,category,lownormalvalue,highnormalvalue
0,10264572,25328682.0,31523226.0,ED,Emergency Department,2110-02-17 00:03:00,2110-02-17 03:46:00,all_stays Emergency Department,3,Transfer,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,10264572,25328682.0,34149732.0,admit,Trauma SICU (TSICU),2110-02-17 03:46:00,2110-02-21 03:24:17,all_stays Trauma SICU (TSICU),22,Transfer,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,10264572,25328682.0,35175138.0,transfer,Vascular,2110-02-21 03:24:17,2110-03-01 14:18:25,all_stays Vascular,24,Transfer,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,10264572,25328682.0,30570317.0,transfer,Vascular,2110-03-01 14:18:25,2110-03-04 14:17:51,all_stays Vascular,24,Transfer,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,10264572,25328682.0,37295469.0,discharge,NaN,2110-03-04 14:17:51,NaN,all_stays nan,25,Transfer,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [126]:
print(complete_dataset['value'].isnull().values.any())
print(complete_dataset['value_categorical'].isnull().values.any())

True
False


In [127]:
complete_dataset[complete_dataset['value'].isnull()]

,subject_id,hadm_id,transfer_id,eventtype,careunit,intime,outtime,value,value_categorical,param_type,...,itemid,valuenum,valueuom,warning,label,abbreviation,linksto,category,lownormalvalue,highnormalvalue
1813156,10862862,20640606.0,NaN,NaN,NaN,NaN,NaN,NaN,text 5629,Text,...,224860.0,NaN,NaN,0.0,Skin Condition,Skin Condition,chartevents,Skin - Assessment,NaN,NaN
1813352,10862862,20640606.0,NaN,NaN,NaN,NaN,NaN,NaN,text 5629,Text,...,224860.0,NaN,NaN,0.0,Skin Condition,Skin Condition,chartevents,Skin - Assessment,NaN,NaN
2907365,10227155,27191372.0,NaN,NaN,NaN,NaN,NaN,NaN,text 3995,Text,...,229534.0,NaN,NaN,0.0,Multi Lumen - Obturator in place,Multi Lumen - Obturator in place,chartevents,Access Lines - Invasive,NaN,NaN
9383889,10670085,26051661.0,NaN,NaN,NaN,NaN,NaN,NaN,text 5629,Text,...,224860.0,NaN,NaN,0.0,Skin Condition,Skin Condition,chartevents,Skin - Assessment,NaN,NaN


In [38]:
complete_dataset[complete_dataset['value_categorical'] == 0]

,subject_id,hadm_id,transfer_id,eventtype,careunit,intime,outtime,value,value_categorical,param_type,...,itemid,valuenum,valueuom,warning,label,abbreviation,linksto,category,lownormalvalue,highnormalvalue
56,10438253,29269604.0,33990465.0,transfer,Cardiac Surgery,2112-05-05 21:39:47,2112-05-07 15:17:07,all_stays Cardiac Surgery,0,Transfer,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
337,10600153,28179068.0,31494598.0,transfer,Cardiac Surgery,2119-01-03 17:41:06,2119-01-06 16:47:22,all_stays Cardiac Surgery,0,Transfer,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
341,10600153,28179068.0,38360765.0,transfer,Cardiac Surgery,2119-01-08 18:45:47,2119-01-10 10:53:19,all_stays Cardiac Surgery,0,Transfer,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
342,10600153,28179068.0,39943315.0,transfer,Cardiac Surgery,2119-01-10 10:53:19,2119-01-14 14:36:16,all_stays Cardiac Surgery,0,Transfer,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
440,10193074,22392305.0,32130736.0,transfer,Cardiac Surgery,2121-02-15 15:25:47,2121-02-16 20:09:37,all_stays Cardiac Surgery,0,Transfer,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ad769cb7-9f7e8b93-66aa638b-7df82431-9fd93386,19157548,NaN,NaN,NaN,NaN,NaN,NaN,photo ABD PORT LINE/TUBE PLACEMENT 1 EXAM,0,ED photo,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1813156,10862862,20640606.0,NaN,NaN,NaN,NaN,NaN,NaN,0,Text,...,224860.0,NaN,NaN,0.0,Skin Condition,Skin Condition,chartevents,Skin - Assessment,NaN,NaN
1813352,10862862,20640606.0,NaN,NaN,NaN,NaN,NaN,NaN,0,Text,...,224860.0,NaN,NaN,0.0,Skin Condition,Skin Condition,chartevents,Skin - Assessment,NaN,NaN
2907365,10227155,27191372.0,NaN,NaN,NaN,NaN,NaN,NaN,0,Text,...,229534.0,NaN,NaN,0.0,Multi Lumen - Obturator in place,Multi Lumen - Obturator in place,chartevents,Access Lines - Invasive,NaN,NaN


## Remodel ICU binned dataset

In [103]:
icu_binned_no_text = pd.read_csv('./dataset_files/icu_binned_no_text_only.csv', index_col=0)
icu_binned_text = pd.read_csv('./dataset_files/text_df.csv', index_col=0)

/var/folders/lk/50s6d7zj3ml029c5x4nygd740000gn/T/ipykernel_87759/1697701215.py:1: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  icu_binned_no_text = pd.read_csv('./dataset_files/icu_binned_no_text_only.csv', index_col=0)


## Prepend 'text' to value categorical of text df

In [104]:
icu_binned_text['value_categorical'] = 'text ' + icu_binned_text['value_categorical'].astype(str)


In [107]:
icu_binned_text.head()

,subject_id,hadm_id,stay_id,charttime,storetime,itemid,value,valuenum,valueuom,warning,label,abbreviation,linksto,category,param_type,lownormalvalue,highnormalvalue,value_categorical
9,10582595,20690213,33235984,2110-02-09 21:46:00,2110-02-09 21:51:00,225090,No,NaN,NaN,0,Interpreter,Interpreter,chartevents,Adm History/FHPA,Text,NaN,NaN,text 3760
10,10582595,20690213,33235984,2110-02-09 21:46:00,2110-02-09 21:51:00,225120,Good,NaN,NaN,0,Appetite,Appetite,chartevents,Adm History/FHPA,Text,NaN,NaN,text 414
14,10582595,20690213,33235984,2110-02-09 21:46:00,2110-02-09 21:51:00,225072,Lives alone,NaN,NaN,0,Living situation,Living situation,chartevents,Adm History/FHPA,Text,NaN,NaN,text 3903
17,10582595,20690213,33235984,2110-02-09 22:00:00,2110-02-10 00:43:00,224650,None,NaN,NaN,0,Ectopy Type 1,Ectopy Type 1,chartevents,Routine Vital Signs,Text,NaN,NaN,text 1111
26,10582595,20690213,33235984,2110-02-09 22:19:00,2110-02-09 22:19:00,224642,Oral,NaN,NaN,0,Temperature Site,Temp Site,chartevents,Routine Vital Signs,Text,NaN,NaN,text 5963


## Merge text and non-text df

In [106]:
icu_binned = pd.concat([icu_binned_no_text, icu_binned_text])

In [111]:
icu_binned['charttime'] = pd.to_datetime(
    icu_binned['charttime'], format='%Y-%m-%d')
icu_binned.head()


,subject_id,hadm_id,stay_id,charttime,storetime,itemid,value,valuenum,valueuom,warning,label,abbreviation,linksto,category,param_type,lownormalvalue,highnormalvalue,value_categorical
0,10582595,20690213,33235984,2110-02-09 20:42:00,2110-02-10 16:45:00,226512,54.3,54.3,kg,0,Admission Weight (Kg),Admission Weight (Kg),chartevents,General,Numeric,NaN,NaN,"Admission Weight (Kg)(53.75148988012528, 58.23..."
7227,10264572,25328682,34149732,2110-02-17 01:40:00,2110-02-17 03:45:00,226512,78.0,78.0,kg,0,Admission Weight (Kg),Admission Weight (Kg),chartevents,General,Numeric,NaN,NaN,"Admission Weight (Kg)(76.14794399684415, 80.62..."
12934,10617255,21586668,37228716,2110-06-17 23:30:00,2110-06-18 01:14:00,226512,61.1,61.1,kg,0,Admission Weight (Kg),Admission Weight (Kg),chartevents,General,Numeric,NaN,NaN,"Admission Weight (Kg)(58.23078070346905, 62.71..."
13690,10499563,27896354,32084946,2110-11-09 06:30:00,2110-11-09 07:30:00,226512,128.0,128.0,kg,0,Admission Weight (Kg),Admission Weight (Kg),chartevents,General,Numeric,NaN,NaN,"Admission Weight (Kg)(125.42014305362565, 129...."
16190,10168722,25403630,32404106,2110-11-14 18:21:00,2110-11-14 21:18:00,226512,53.6,53.6,kg,0,Admission Weight (Kg),Admission Weight (Kg),chartevents,General,Numeric,NaN,NaN,"Admission Weight (Kg)(49.27219905678151, 53.75..."


# Merge all dataframes

In [140]:
complete_data = pd.concat([icu_binned, metadata, all_stays])
complete_data.head()

,subject_id,hadm_id,stay_id,charttime,storetime,itemid,value,valuenum,valueuom,warning,...,StudyDate,StudyTime,ProcedureCodeSequence_CodeMeaning,ViewCodeSequence_CodeMeaning,PatientOrientationCodeSequence_CodeMeaning,transfer_id,eventtype,careunit,intime,outtime
0,10582595,20690213.0,33235984.0,2110-02-09 20:42:00,2110-02-10 16:45:00,226512.0,54.3,54.3,kg,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7227,10264572,25328682.0,34149732.0,2110-02-17 01:40:00,2110-02-17 03:45:00,226512.0,78.0,78.0,kg,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12934,10617255,21586668.0,37228716.0,2110-06-17 23:30:00,2110-06-18 01:14:00,226512.0,61.1,61.1,kg,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13690,10499563,27896354.0,32084946.0,2110-11-09 06:30:00,2110-11-09 07:30:00,226512.0,128.0,128.0,kg,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16190,10168722,25403630.0,32404106.0,2110-11-14 18:21:00,2110-11-14 21:18:00,226512.0,53.6,53.6,kg,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [141]:
complete_data['value_categorical'] = complete_data['value_categorical'].astype('category')
complete_data.dtypes

subject_id                                             int64
hadm_id                                              float64
stay_id                                              float64
charttime                                     datetime64[ns]
storetime                                             object
itemid                                               float64
value                                                 object
valuenum                                             float64
valueuom                                              object
warning                                              float64
label                                                 object
abbreviation                                          object
linksto                                               object
category                                              object
param_type                                            object
lownormalvalue                                       float64
highnormalvalue         

In [142]:
complete_data['value_categorical'] = complete_data['value_categorical'].cat.codes


In [143]:
get_distribution(complete_data)

100%|██████████| 16865/16865 [02:22<00:00, 118.62it/s]
0it [00:00, ?it/s]

Multiple values: 0
Single values: 16865 
Value types: []


In [152]:
complete_data['hadm_id'].isnull().values.any()

True

In [160]:
complete_data = complete_data.sort_values(by=['charttime'])
complete_data

,subject_id,hadm_id,stay_id,charttime,storetime,itemid,value,valuenum,valueuom,warning,...,StudyDate,StudyTime,ProcedureCodeSequence_CodeMeaning,ViewCodeSequence_CodeMeaning,PatientOrientationCodeSequence_CodeMeaning,transfer_id,eventtype,careunit,intime,outtime
c2a3cbe8-ad80fc7b-9696e471-e41fb7a7-b0a2d9d9,17195991,NaN,NaN,2110-01-11 00:00:00,NaN,NaN,photo CHEST (PORTABLE AP),NaN,NaN,NaN,...,21100111.0,220111.359,CHEST (PORTABLE AP),antero-posterior,Erect,NaN,NaN,NaN,NaN,NaN
1b9352eb-9ae051ae-8fd2c6ad-165db0b8-e74211a2,17195991,NaN,NaN,2110-01-12 00:00:00,NaN,NaN,photo CHEST (PORTABLE AP),NaN,NaN,NaN,...,21100112.0,110815.593,CHEST (PORTABLE AP),antero-posterior,NaN,NaN,NaN,NaN,NaN,NaN
06980cc1-94b37c18-8297f246-90ffe664-bc51fd35,17195991,NaN,NaN,2110-01-13 00:00:00,NaN,NaN,photo CHEST (PORTABLE AP),NaN,NaN,NaN,...,21100113.0,92700.921,CHEST (PORTABLE AP),antero-posterior,Erect,NaN,NaN,NaN,NaN,NaN
43520d74-e2328ddf-3ee9c45e-d08e6e10-c999234c,17195991,NaN,NaN,2110-01-14 00:00:00,NaN,NaN,photo CHEST (PORTABLE AP),NaN,NaN,NaN,...,21100114.0,40913.203,CHEST (PORTABLE AP),antero-posterior,Erect,NaN,NaN,NaN,NaN,NaN
b865264e-a6737a8e-55666fa7-1fcf16ed-bbf5df04,17195991,NaN,NaN,2110-01-14 00:00:00,NaN,NaN,photo CHEST (PORTABLE AP),NaN,NaN,NaN,...,21100114.0,40913.203,CHEST (PORTABLE AP),antero-posterior,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9720589,10488182,22756387.0,32087485.0,2209-06-06 03:00:00,2209-06-06 04:05:00,220045.0,122.0,122.0,bpm,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9720591,10488182,22756387.0,32087485.0,2209-06-06 03:00:00,2209-06-06 04:05:00,220210.0,15.0,15.0,insp/min,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9720588,10488182,22756387.0,32087485.0,2209-06-06 03:00:00,2209-06-06 04:05:00,224650.0,PAC's,NaN,NaN,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9720590,10488182,22756387.0,32087485.0,2209-06-06 03:00:00,2209-06-06 04:05:00,226480.0,Occasional,NaN,NaN,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [161]:
complete_data.to_csv('complete_data_cleaned.csv')

In [ ]:
save_as_pickle(complete_data, 'complete_data_cleaned_v1')

## Aux function for storing and retrieving pickles

In [2]:
def save_as_pickle(data, file_name):
    file = open(file_name, 'wb')
    pickle.dump(data, file)
    file.close()
    
def get_pickle(file_name):
    return pickle.load(open(file_name, 'rb'))

## Check labels

In [62]:
labels = get_pickle('./labels')

In [63]:
types = [{
    'label': x['measurement'],  
    'type': 'ICU',
    'encodings': list(dict.fromkeys([y['value_enc'] for y in x['values'] if y['value_enc'] ]))
    } if x['type'] == 'ICU measurement' else {
        'type': "Transfer",
        'encodings': list(dict.fromkeys([y['value_enc'] for y in x['values'] if y['value_enc']]))
} if x['type'] == 'Transfer' else {
        'type': "Photo",
        'encodings': list(dict.fromkeys([y['value_enc'] for y in x['values'] if y['value_enc']]))
} for x in labels]
types

[{'type': 'Transfer', 'encodings': []},
 {'type': 'Transfer', 'encodings': [10524]},
 {'type': 'Transfer', 'encodings': [10520]},
 {'type': 'Transfer', 'encodings': [10522]},
 {'type': 'Transfer', 'encodings': [10530]},
 {'type': 'Transfer', 'encodings': [10519]},
 {'type': 'Transfer', 'encodings': [10529]},
 {'type': 'Transfer', 'encodings': [10508]},
 {'type': 'Transfer', 'encodings': [10509]},
 {'type': 'Transfer', 'encodings': [10506]},
 {'type': 'Transfer', 'encodings': [10528]},
 {'type': 'Transfer', 'encodings': [10516]},
 {'type': 'Transfer', 'encodings': [10525]},
 {'type': 'Transfer', 'encodings': [10517]},
 {'type': 'Transfer', 'encodings': [10505]},
 {'type': 'Transfer', 'encodings': [10518]},
 {'type': 'Transfer', 'encodings': [10513]},
 {'type': 'Transfer', 'encodings': [10527]},
 {'type': 'Transfer', 'encodings': [10526]},
 {'type': 'Transfer', 'encodings': [10512]},
 {'type': 'Transfer', 'encodings': [10511]},
 {'type': 'Transfer', 'encodings': [10510]},
 {'type': 'Tran

In [64]:
types = [{'label': x['measurement'],
          'encodings': [y['value_enc'] if x['type'] == 'ICU measurement' else {} for y in x['values']]} for x in labels]

print(types)

KeyError: 'measurement'

In [30]:
labels[len(labels) - 1]['measurement'] = 'Not available'
print(F"last label: {labels[len(labels) -1 ]}")


last label: {'type': 'ICU measurement', 'measurement': 'Not available', 'values': []}


In [65]:
labels[0]

{'type': 'Transfer', 'care_unit': nan, 'values': []}

In [75]:
save_as_pickle(labels, 'labels')


In [ ]:
all_stays[all_stays['careunit'].isnull()]

In [124]:
changed = False

if 'care_unit' in labels[0] and (type(labels[0]['care_unit']) == 'int' and math.isnan(labels[0]['care_unit'])):
    labels[0]['care_unit'] = 'Not available'
    print(labels[0])
    changed = True
    
if 'measurement' in labels[len(labels) - 1] and type(labels[len(labels) - 1]) == 'int' and math.isnan(labels[len(labels) - 1]['measurement']):
    labels[len(labels) - 1]['measurement'] = "Not available"
    print(labels[len(labels) - 1])
    changed = True

            
for i in range(0, len(labels)):
    if ('type' in labels[i]) and (labels[i]['type'] == 'Photo') and (not isinstance(labels[i]['viewpoint'], str)):
        labels[i]['viewpoint'] = 'Not available'
        print(labels[i])
        changed = True

    

if changed:
    print('saving...')
    save_as_pickle(labels, 'labels')


{'type': 'Photo', 'viewpoint': 'Not available', 'values': []}
saving...


In [123]:
# math.isnan(labels[0]['care_unit'])
# 'measurement' in labels[len(labels) - 1]
# labels[len(labels) - 1]['measurement']

for i in range(0, len(labels)):
    if ('type' in labels[i] )and (labels[i]['type'] == 'Photo') and (not isinstance(labels[i]['viewpoint'], str)):
        print(f"Found at index: {i}\nItem: {labels[i]}")
        labels[i]['viewpoint'] = 'Not available'

Found at index: 26
Item: {'type': 'Photo', 'viewpoint': nan, 'values': []}
